In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

def read_csv_file(spark, path):
    """Reads a CSV file into a Spark DataFrame"""
    return (
        spark.read
            .option("header", True)
            .option("inferSchema", True)
            .csv(path)
    )

def write_df_to_csv(df, output_path):
    """Writes a DataFrame to a DBFS path as CSV"""
    (
        df.repartition(1)
          .write
          .mode("overwrite")
          .option("header", True)
          .csv(output_path)
    )
    print(f"✅ CSV written to {output_path}")

def camel_to_snake(name):
    """Convert CamelCase column name to snake_case"""
    import re
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

def convert_camel_to_snake_udf(df):
    """Renames all columns from CamelCase to snake_case"""
    for col in df.columns:
        df = df.withColumnRenamed(col, camel_to_snake(col))
    return df

def add_load_date(df, column_name="load_date"):
    """Adds load_date column with current date"""
    return df.withColumn(column_name, F.current_date())

# Writing DataFrame to Delta
def write_delta(df, db_name, table_name, mode="overwrite"):
    """Writes a DataFrame as Delta Table"""
    (
        df.write
          .format("delta")
          .mode(mode)
          .saveAsTable(f"{db_name}.{table_name}")
    )
    
    print(f"✅ Delta table written to {db_name}.{table_name}")